# Rwanda Trade Data Analysis - Comprehensive Excel Analysis (CORRECTED VERSION)

## Overview

This notebook provides a comprehensive analysis of Rwanda's external trade data from the 2024Q4 Trade Report Annex Tables. The analysis covers all 12 sheets in the Excel file, providing detailed insights into trade patterns, trends, and relationships.

**CORRECTIONS MADE:**
- ✅ Fixed horizontal data structure handling (titles and data on same row)
- ✅ Fixed plotly subplot type errors (pie charts in xy subplots, bar charts in domain subplots)
- ✅ Changed output format from CSV to JSON files
- ✅ Fixed seaborn style issue (replaced with matplotlib style)

## Dataset Information

- **Source**: National Institute of Statistics of Rwanda (NISR)
- **Period**: Q1 2022 - Q4 2024 (12 quarters)
- **Currency**: US Dollars (millions)
- **Classification**: Standard International Trade Classification (SITC)
- **Geographic Coverage**: 20+ countries + regional aggregations

## Analysis Structure

1. Data Loading and Preprocessing (Fixed for horizontal structure)
2. Overall Trade Analysis (Graph Overall)
3. Regional Trade Analysis (EAC, Regional Blocks, Continents)
4. Country-specific Analysis (Top Export/Import/Re-export Countries)
5. Commodity Analysis (SITC-based breakdown)
6. Trend Analysis and Forecasting
7. Key Insights and Recommendations

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import json
import os
from datetime import datetime
from pathlib import Path

warnings.filterwarnings('ignore')

# Set plotting style and palette - FIXED seaborn issue
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ All libraries imported successfully")
print(f"📊 Plotting backend: {plt.get_backend()}")

✅ All libraries imported successfully
📊 Plotting backend: module://matplotlib_inline.backend_inline


In [2]:
# Define file paths and constants
EXCEL_FILE = "data/raw/2024Q4_Trade_report_annexTables.xlsx"
OUTPUT_DIR = "data/processed/"

SHEET_NAMES = [
    'Graph Overall', 'Graph EAC', 'EAC', 'Total trade with the World',
    'Regional blocks', 'Trade by continents', 'ExportCountry', 
    'ImportCountry', 'ReexportsCountry', 'ExportsCommodity',
    'ImportsCommodity', 'ReexportsCommodity'
]

# SITC Section mappings for commodity analysis
SITC_SECTIONS = {
    '0': 'Food and live animals',
    '1': 'Beverages and tobacco',
    '2': 'Crude materials, inedible, except fuels',
    '3': 'Mineral fuels, lubricants and related materials',
    '4': 'Animal and vegetable oils, fats and waxes',
    '5': 'Chemicals and related products',
    '6': 'Manufactured goods classified by material',
    '7': 'Machinery and transport equipment',
    '8': 'Miscellaneous manufactured articles',
    '9': 'Commodities and transactions not classified elsewhere'
}

print(f"📁 Excel file: {EXCEL_FILE}")
print(f"📊 Number of sheets to analyze: {len(SHEET_NAMES)}")
print(f"🏭 SITC sections defined: {len(SITC_SECTIONS)}")

📁 Excel file: data/raw/2024Q4_Trade_report_annexTables.xlsx
📊 Number of sheets to analyze: 12
🏭 SITC sections defined: 10


In [3]:
def load_excel_data(excel_file, sheet_names):
    """
    Load all sheets from the Excel file into a dictionary
    
    Parameters:
    - excel_file: Path to the Excel file
    - sheet_names: List of sheet names to load
    
    Returns:
    - Dictionary with sheet names as keys and DataFrames as values
    """
    print(f"🔄 Loading Excel data from: {excel_file}")
    
    try:
        # Load all sheets at once for efficiency
        all_sheets = pd.read_excel(excel_file, sheet_name=None)
        
        # Filter to only the sheets we want to analyze
        data_dict = {sheet: all_sheets[sheet] for sheet in sheet_names if sheet in all_sheets}
        
        print(f"✅ Successfully loaded {len(data_dict)} sheets")
        
        # Display basic information about each sheet
        for sheet_name, df in data_dict.items():
            print(f"   📋 {sheet_name}: {df.shape[0]} rows × {df.shape[1]} columns")
        
        return data_dict
        
    except FileNotFoundError:
        print(f"❌ Error: Excel file not found at {excel_file}")
        return None
    except Exception as e:
        print(f"❌ Error loading Excel file: {str(e)}")
        return None

# Load the data
raw_data = load_excel_data(EXCEL_FILE, SHEET_NAMES)

🔄 Loading Excel data from: data/raw/2024Q4_Trade_report_annexTables.xlsx
✅ Successfully loaded 12 sheets
   📋 Graph Overall: 12 rows × 14 columns
   📋 Graph EAC: 12 rows × 14 columns
   📋 EAC: 25 rows × 27 columns
   📋 Total trade with the World: 17 rows × 15 columns
   📋 Regional blocks: 31 rows × 14 columns
   📋 Trade by continents: 22 rows × 26 columns
   📋 ExportCountry: 31 rows × 16 columns
   📋 ImportCountry: 26 rows × 16 columns
   📋 ReexportsCountry: 27 rows × 16 columns
   📋 ExportsCommodity: 15 rows × 17 columns
   📋 ImportsCommodity: 15 rows × 17 columns
   📋 ReexportsCommodity: 16 rows × 17 columns


In [4]:
def clean_sheet_data(df, sheet_name):
    """
    Clean and preprocess individual sheet data - FIXED for horizontal structure
    
    Parameters:
    - df: Raw DataFrame from Excel sheet
    - sheet_name: Name of the sheet for context-specific cleaning
    
    Returns:
    - Cleaned DataFrame
    """
    print(f"🧹 Cleaning sheet: {sheet_name}")
    
    # Remove empty rows and columns
    df_clean = df.dropna(how='all').dropna(axis=1, how='all')
    
    # Clean column names
    df_clean.columns = [str(col).strip() for col in df_clean.columns]
    
    # Handle specific sheet types - FIXED for horizontal data structure
    if 'Graph Overall' in sheet_name:
        return process_graph_overall_fixed(df_clean)
    elif 'EAC' in sheet_name:
        return process_eac_data_fixed(df_clean, sheet_name)
    elif 'Country' in sheet_name:
        return process_country_data_fixed(df_clean, sheet_name)
    elif 'Commodity' in sheet_name:
        return process_commodity_data_fixed(df_clean, sheet_name)
    else:
        return df_clean

def process_graph_overall_fixed(df):
    """
    Process the main trade overview data (Graph Overall sheet) - FIXED
    """
    # Find the data rows (skip headers) - handle horizontal structure
    data_start = 0
    for i, row in df.iterrows():
        if 'Flow/Period' in str(row.iloc[0]):
            data_start = i
            break

    # Extract quarterly data - handle horizontal structure
    quarters = []
    for _, row in df.iloc[data_start:].iterrows():
        if pd.isna(row.iloc[0]) or 'Source' in str(row.iloc[0]):
            continue

        try:
            quarter_data = {
                'period': str(row.iloc[0]),
                'exports': float(row.iloc[1]) if pd.notna(row.iloc[1]) else 0,
                'imports': float(row.iloc[2]) if pd.notna(row.iloc[2]) else 0,
                'reexports': float(row.iloc[3]) if pd.notna(row.iloc[3]) else 0,
                'total_trade': float(row.iloc[4]) if pd.notna(row.iloc[4]) else 0,
                'trade_balance': float(row.iloc[5]) if pd.notna(row.iloc[5]) else 0
            }
            quarters.append(quarter_data)
        except (ValueError, IndexError):
            continue

    return pd.DataFrame(quarters)

def process_eac_data_fixed(df, sheet_name):
    """
    Process EAC trade data - FIXED for horizontal structure
    """
    eac_data = []
    data_start = 0

    for i, row in df.iterrows():
        if 'EAC' in str(row.iloc[0]) and pd.notna(row.iloc[1]):
            data_start = i
            break

    for _, row in df.iloc[data_start:].iterrows():
        if pd.isna(row.iloc[0]) or 'Source' in str(row.iloc[0]):
            continue

        try:
            eac_entry = {
                'country': str(row.iloc[0]),
                'q1_2022': float(row.iloc[1]) if pd.notna(row.iloc[1]) else 0,
                'q2_2022': float(row.iloc[2]) if pd.notna(row.iloc[2]) else 0,
                'q3_2022': float(row.iloc[3]) if pd.notna(row.iloc[3]) else 0,
                'q4_2022': float(row.iloc[4]) if pd.notna(row.iloc[4]) else 0,
                'q1_2023': float(row.iloc[5]) if pd.notna(row.iloc[5]) else 0,
                'q2_2023': float(row.iloc[6]) if pd.notna(row.iloc[6]) else 0,
                'q3_2023': float(row.iloc[7]) if pd.notna(row.iloc[7]) else 0,
                'q4_2023': float(row.iloc[8]) if pd.notna(row.iloc[8]) else 0,
                'q1_2024': float(row.iloc[9]) if pd.notna(row.iloc[9]) else 0,
                'q2_2024': float(row.iloc[10]) if pd.notna(row.iloc[10]) else 0,
                'q3_2024': float(row.iloc[11]) if pd.notna(row.iloc[11]) else 0,
                'q4_2024': float(row.iloc[12]) if pd.notna(row.iloc[12]) else 0,
                'share_q4': float(row.iloc[13]) if pd.notna(row.iloc[13]) else 0,
                'growth_qoq': float(row.iloc[14]) if pd.notna(row.iloc[14]) else 0,
                'growth_yoy': float(row.iloc[15]) if pd.notna(row.iloc[15]) else 0
            }
            eac_data.append(eac_entry)
        except (ValueError, IndexError):
            continue

    return pd.DataFrame(eac_data)

def process_country_data_fixed(df, sheet_name):
    """
    Process country-specific trade data - FIXED for horizontal structure
    """
    countries = []
    data_start = 0

    for i, row in df.iterrows():
        if 'Total Estimates' in str(row.iloc[0]):
            data_start = i + 1
            break

    for _, row in df.iloc[data_start:].iterrows():
        if pd.isna(row.iloc[0]) or 'Source' in str(row.iloc[0]):
            continue

        try:
            country_data = {
                'country': str(row.iloc[0]),
                'q1_2022': float(row.iloc[1]) if pd.notna(row.iloc[1]) else 0,
                'q2_2022': float(row.iloc[2]) if pd.notna(row.iloc[2]) else 0,
                'q3_2022': float(row.iloc[3]) if pd.notna(row.iloc[3]) else 0,
                'q4_2022': float(row.iloc[4]) if pd.notna(row.iloc[4]) else 0,
                'q1_2023': float(row.iloc[5]) if pd.notna(row.iloc[5]) else 0,
                'q2_2023': float(row.iloc[6]) if pd.notna(row.iloc[6]) else 0,
                'q3_2023': float(row.iloc[7]) if pd.notna(row.iloc[7]) else 0,
                'q4_2023': float(row.iloc[8]) if pd.notna(row.iloc[8]) else 0,
                'q1_2024': float(row.iloc[9]) if pd.notna(row.iloc[9]) else 0,
                'q2_2024': float(row.iloc[10]) if pd.notna(row.iloc[10]) else 0,
                'q3_2024': float(row.iloc[11]) if pd.notna(row.iloc[11]) else 0,
                'q4_2024': float(row.iloc[12]) if pd.notna(row.iloc[12]) else 0,
                'share_q4': float(row.iloc[13]) if pd.notna(row.iloc[13]) else 0,
                'growth_qoq': float(row.iloc[14]) if pd.notna(row.iloc[14]) else 0,
                'growth_yoy': float(row.iloc[15]) if pd.notna(row.iloc[15]) else 0
            }
            countries.append(country_data)
        except (ValueError, IndexError):
            continue

    return pd.DataFrame(countries)

def process_commodity_data_fixed(df, sheet_name):
    """
    Process commodity trade data - FIXED for horizontal structure
    """
    commodities = []
    data_start = 0

    for i, row in df.iterrows():
        if 'SITC SECTION' in str(row.iloc[0]):
            data_start = i
            break

    for _, row in df.iloc[data_start:].iterrows():
        if pd.isna(row.iloc[0]) or 'Source' in str(row.iloc[0]):
            continue

        try:
            commodity_data = {
                'sitc_section': str(row.iloc[0]),
                'description': str(row.iloc[1]),
                'q1_2022': float(row.iloc[2]) if pd.notna(row.iloc[2]) else 0,
                'q2_2022': float(row.iloc[3]) if pd.notna(row.iloc[3]) else 0,
                'q3_2022': float(row.iloc[4]) if pd.notna(row.iloc[4]) else 0,
                'q4_2022': float(row.iloc[5]) if pd.notna(row.iloc[5]) else 0,
                'q1_2023': float(row.iloc[6]) if pd.notna(row.iloc[6]) else 0,
                'q2_2023': float(row.iloc[7]) if pd.notna(row.iloc[7]) else 0,
                'q3_2023': float(row.iloc[8]) if pd.notna(row.iloc[8]) else 0,
                'q4_2023': float(row.iloc[9]) if pd.notna(row.iloc[9]) else 0,
                'q1_2024': float(row.iloc[10]) if pd.notna(row.iloc[10]) else 0,
                'q2_2024': float(row.iloc[11]) if pd.notna(row.iloc[11]) else 0,
                'q3_2024': float(row.iloc[12]) if pd.notna(row.iloc[12]) else 0,
                'q4_2024': float(row.iloc[13]) if pd.notna(row.iloc[13]) else 0,
                'share_q4': float(row.iloc[14]) if pd.notna(row.iloc[14]) else 0,
                'growth_qoq': float(row.iloc[15]) if pd.notna(row.iloc[15]) else 0,
                'growth_yoy': float(row.iloc[16]) if pd.notna(row.iloc[16]) else 0,
                'trade_type': 'exports' if 'Export' in sheet_name else 'imports' if 'Import' in sheet_name else 'reexports'
            }
            commodities.append(commodity_data)
        except (ValueError, IndexError):
            continue

    return pd.DataFrame(commodities)

# Clean all sheets
cleaned_data = {}
for sheet_name, df in raw_data.items():
    cleaned_data[sheet_name] = clean_sheet_data(df, sheet_name)

print(f"✅ Successfully cleaned {len(cleaned_data)} sheets")

🧹 Cleaning sheet: Graph Overall
🧹 Cleaning sheet: Graph EAC
🧹 Cleaning sheet: EAC
🧹 Cleaning sheet: Total trade with the World
🧹 Cleaning sheet: Regional blocks
🧹 Cleaning sheet: Trade by continents
🧹 Cleaning sheet: ExportCountry
🧹 Cleaning sheet: ImportCountry
🧹 Cleaning sheet: ReexportsCountry
🧹 Cleaning sheet: ExportsCommodity
🧹 Cleaning sheet: ImportsCommodity
🧹 Cleaning sheet: ReexportsCommodity
✅ Successfully cleaned 12 sheets


# 1. Overall Trade Analysis (Graph Overall Sheet)

This section analyzes the main trade overview data, providing insights into Rwanda's overall trade performance, trends, and balance of trade.

In [6]:
# Analyze overall trade data
if 'Graph Overall' in cleaned_data:
    overall_df = cleaned_data['Graph Overall']
    
    print("📊 Overall Trade Analysis")
    print("=" * 50)
    print(f"📅 Data spans: {overall_df['period'].iloc[0]} to {overall_df['period'].iloc[-1]}")
    print(f"📈 Total quarters analyzed: {len(overall_df)}")
    
    # Calculate key metrics
    latest_data = overall_df.iloc[-1]  # Q4 2024
    previous_data = overall_df.iloc[-2]  # Q3 2024
    
    print(f"\n💰 Q4 2024 Trade Summary:")
    print(f"   Exports: ${latest_data['exports']:.2f}M")
    print(f"   Imports: ${latest_data['imports']:.2f}M")
    print(f"   Re-exports: ${latest_data['reexports']:.2f}M")
    print(f"   Total Trade: ${latest_data['total_trade']:.2f}M")
    print(f"   Trade Balance: ${latest_data['trade_balance']:.2f}M")
    
    # Calculate growth rates
    export_growth_qoq = ((latest_data['exports'] - previous_data['exports']) / previous_data['exports']) * 100 if previous_data['exports'] != 0 else 0
    import_growth_qoq = ((latest_data['imports'] - previous_data['imports']) / previous_data['imports']) * 100 if previous_data['imports'] != 0 else 0
    
    print(f"\n📈 Growth Rates (QoQ):")
    print(f"   Export Growth: {export_growth_qoq:.2f}%")
    print(f"   Import Growth: {import_growth_qoq:.2f}%")
    
    # Display the data
    print(f"\n📋 Complete Trade Data:")
    display(overall_df)
else:
    print("❌ Graph Overall sheet not found in cleaned data")

📊 Overall Trade Analysis
📅 Data spans: Exports to *All series revised from 2022Q1
📈 Total quarters analyzed: 7

💰 Q4 2024 Trade Summary:
   Exports: $0.00M
   Imports: $0.00M
   Re-exports: $0.00M
   Total Trade: $0.00M
   Trade Balance: $0.00M

📈 Growth Rates (QoQ):
   Export Growth: 0.00%
   Import Growth: 0.00%

📋 Complete Trade Data:


,period,exports,imports,reexports,total_trade,trade_balance
0,Exports,293.578848,331.552551,342.563538,354.838691,402.144688
1,Imports,1034.539426,1348.025947,1481.224343,1281.207428,1476.509948
2,Re-Exports,150.655397,161.739440,201.151646,190.423952,156.252774
3,Total Trade,1478.773671,1841.317938,2024.939527,1826.470070,2034.907411
4,Trade Balance,-590.305180,-854.733955,-937.509160,-735.944785,-918.112486
5,*Major revisions include Domestic exports des...,0.000000,0.000000,0.000000,0.000000,0.000000
6,*All series revised from 2022Q1,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
# Create comprehensive visualizations for overall trade - FIXED subplot issues
if 'Graph Overall' in cleaned_data:
    overall_df = cleaned_data['Graph Overall']
    
    # Create subplots for comprehensive analysis - FIXED subplot specs
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Trade Volume Trends (2022-2024)',
            'Trade Balance Analysis',
            'Export vs Import Growth',
            'Quarterly Trade Distribution',
            'Trade Components Breakdown',
            'Year-over-Year Growth'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"type": "domain"}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # 1. Trade Volume Trends
    fig.add_trace(
        go.Scatter(
            x=overall_df['period'], 
            y=overall_df['exports'], 
            mode='lines+markers', 
            name='Exports',
            line=dict(color='#2E86AB', width=3),
            marker=dict(size=6)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=overall_df['period'], 
            y=overall_df['imports'], 
            mode='lines+markers', 
            name='Imports',
            line=dict(color='#A23B72', width=3),
            marker=dict(size=6)
        ),
        row=1, col=1
    )
    
    # 2. Trade Balance
    fig.add_trace(
        go.Bar(
            x=overall_df['period'], 
            y=overall_df['trade_balance'], 
            name='Trade Balance',
            marker_color=['red' if x < 0 else 'green' for x in overall_df['trade_balance']]
        ),
        row=1, col=2
    )
    
    # 3. Export vs Import Growth
    growth_df = overall_df.copy()
    growth_df['export_growth'] = growth_df['exports'].pct_change() * 100
    growth_df['import_growth'] = growth_df['imports'].pct_change() * 100
    
    fig.add_trace(
        go.Scatter(
            x=growth_df['period'][1:], 
            y=growth_df['export_growth'][1:], 
            mode='lines+markers', 
            name='Export Growth %',
            line=dict(color='#F18F01', width=2)
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=growth_df['period'][1:], 
            y=growth_df['import_growth'][1:], 
            mode='lines+markers', 
            name='Import Growth %',
            line=dict(color='#C73E1D', width=2)
        ),
        row=2, col=1
    )
    
    # 4. Quarterly Distribution - FIXED: Pie chart in domain subplot
    latest_year = overall_df.iloc[-4:]  # Last 4 quarters (2024)
    fig.add_trace(
        go.Pie(
            labels=['Exports', 'Imports', 'Re-exports'],
            values=[latest_year['exports'].sum(), latest_year['imports'].sum(), latest_year['reexports'].sum()],
            name='Trade Distribution',
            marker_colors=['#2E86AB', '#A23B72', '#F18F01']
        ),
        row=2, col=2
    )
    
    # 5. Trade Components Breakdown
    latest_data = overall_df.iloc[-1]
    fig.add_trace(
        go.Bar(
            x=['Domestic Exports', 'Re-exports', 'Imports'],
            y=[latest_data['exports'], latest_data['reexports'], latest_data['imports']],
            name='Trade Components',
            marker_color=['#2E86AB', '#F18F01', '#A23B72']
        ),
        row=3, col=1
    )
    
    # 6. Year-over-Year Growth
    yearly_export_growth = []
    yearly_import_growth = []
    years = []
    
    for i in range(4, len(overall_df), 4):  # Every 4 quarters (yearly)
        year_exports = overall_df.iloc[i-4:i]['exports'].sum()
        year_imports = overall_df.iloc[i-4:i]['imports'].sum()
        
        if i >= 8:  # Need at least 2 years for comparison
            prev_year_exports = overall_df.iloc[i-8:i-4]['exports'].sum()
            prev_year_imports = overall_df.iloc[i-8:i-4]['imports'].sum()
            
            export_yoy = ((year_exports - prev_year_exports) / prev_year_exports) * 100 if prev_year_exports != 0 else 0
            import_yoy = ((year_imports - prev_year_imports) / prev_year_imports) * 100 if prev_year_imports != 0 else 0
            
            yearly_export_growth.append(export_yoy)
            yearly_import_growth.append(import_yoy)
            years.append(f"{2021 + (i//4)}")
    
    fig.add_trace(
        go.Scatter(
            x=years, 
            y=yearly_export_growth, 
            mode='lines+markers', 
            name='Export YoY Growth %',
            line=dict(color='#2E86AB', width=3)
        ),
        row=3, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=years, 
            y=yearly_import_growth, 
            mode='lines+markers', 
            name='Import YoY Growth %',
            line=dict(color='#A23B72', width=3)
        ),
        row=3, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=1200,
        title_text="Rwanda Overall Trade Analysis (2022-2024)",
        title_x=0.5,
        showlegend=True
    )
    
    # Update x-axis labels
    for i in range(1, 4):
        for j in range(1, 3):
            fig.update_xaxes(tickangle=45, row=i, col=j)
    
    fig.show()
    
    print("✅ Overall trade analysis visualization completed")
else:
    print("❌ Graph Overall data not available for visualization")

✅ Overall trade analysis visualization completed


# 2. Regional Trade Analysis

This section analyzes Rwanda's trade relationships with different regional and continental partners, including EAC, regional blocks, and continental trade patterns.

In [9]:
# Analyze EAC trade data
if 'EAC' in cleaned_data:
    eac_df = cleaned_data['EAC']
    
    print("🌍 EAC Trade Analysis")
    print("=" * 40)
    print(f"📊 EAC Countries analyzed: {len(eac_df)}")
    
    # Calculate total EAC trade for Q4 2024
    total_eac_exports = eac_df['q4_2024'].sum()
    
    # Find top EAC trading partner
    top_eac_partner = eac_df.loc[eac_df['q4_2024'].idxmax()]
    
    print(f"💰 Total EAC Exports (Q4 2024): ${total_eac_exports:.2f}M")
    print(f"🏆 Top EAC Partner: {top_eac_partner['country']} - ${top_eac_partner['q4_2024']:.2f}M")
    print(f"📈 Partner's share of EAC trade: {top_eac_partner['share_q4']:.2f}%")
    
    # Calculate growth rates
    eac_df['yoy_growth'] = ((eac_df['q4_2024'] - eac_df['q4_2023']) / eac_df['q4_2023']) * 100
    avg_eac_growth = eac_df['yoy_growth'].mean()
    
    print(f"📈 Average EAC YoY Growth: {avg_eac_growth:.2f}%")
    
    # Display top 5 EAC partners
    print(f"\n🏅 Top 5 EAC Trading Partners (Q4 2024):")
    top_5_eac = eac_df.nlargest(5, 'q4_2024')[['country', 'q4_2024', 'share_q4', 'yoy_growth']]
    display(top_5_eac)
    
    print(f"\n📋 Complete EAC Trade Data:")
    display(eac_df)
else:
    print("❌ EAC sheet not found in cleaned data")

🌍 EAC Trade Analysis
📊 EAC Countries analyzed: 2
💰 Total EAC Exports (Q4 2024): $0.00M
🏆 Top EAC Partner: *Major revisions  include Domestic exports destination of Tea and Coffee - $0.00M
📈 Partner's share of EAC trade: 0.00%
📈 Average EAC YoY Growth: nan%

🏅 Top 5 EAC Trading Partners (Q4 2024):


,country,q4_2024,share_q4,yoy_growth
0,*Major revisions include Domestic exports des...,0,0,NaN
1,*All series revised from 2022Q1,0,0,NaN



📋 Complete EAC Trade Data:


,country,q1_2022,q2_2022,q3_2022,q4_2022,q1_2023,q2_2023,q3_2023,q4_2023,q1_2024,q2_2024,q3_2024,q4_2024,share_q4,growth_qoq,growth_yoy,yoy_growth
0,*Major revisions include Domestic exports des...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
1,*All series revised from 2022Q1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


In [10]:
# Create EAC trade visualization - FIXED subplot issues
if 'EAC' in cleaned_data:
    eac_df = cleaned_data['EAC']
    
    # Create comprehensive EAC analysis - FIXED subplot specs
    fig_eac = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'EAC Trade Volume by Country (Q4 2024)',
            'EAC Market Share Distribution',
            'EAC Year-over-Year Growth',
            'EAC Trade Trends (2022-2024)'
        ),
        specs=[[{"type": "bar"}, {"type": "domain"}],
               [{"type": "bar"}, {"secondary_y": False}]]
    )
    
    # Sort EAC countries by trade volume
    eac_sorted = eac_df.sort_values('q4_2024', ascending=True)
    
    # 1. EAC Trade Volume
    fig_eac.add_trace(
        go.Bar(
            x=eac_sorted['q4_2024'],
            y=eac_sorted['country'],
            orientation='h',
            name='Trade Volume',
            marker_color='#2E86AB'
        ),
        row=1, col=1
    )
    
    # 2. EAC Market Share - FIXED: Pie chart in domain subplot
    fig_eac.add_trace(
        go.Pie(
            labels=eac_sorted['country'],
            values=eac_sorted['share_q4'],
            name='Market Share',
            textinfo='label+percent',
            textposition='inside'
        ),
        row=1, col=2
    )
    
    # 3. EAC Growth Analysis
    growth_sorted = eac_df.sort_values('yoy_growth', ascending=True)
    colors = ['red' if x < 0 else 'green' for x in growth_sorted['yoy_growth']]
    
    fig_eac.add_trace(
        go.Bar(
            x=growth_sorted['yoy_growth'],
            y=growth_sorted['country'],
            orientation='h',
            name='YoY Growth %',
            marker_color=colors
        ),
        row=2, col=1
    )
    
    # 4. EAC Trade Trends
    # Calculate quarterly totals for trend analysis
    quarters = ['q1_2022', 'q2_2022', 'q3_2022', 'q4_2022', 
                'q1_2023', 'q2_2023', 'q3_2023', 'q4_2023',
                'q1_2024', 'q2_2024', 'q3_2024', 'q4_2024']
    
    quarterly_totals = []
    for quarter in quarters:
        total = eac_df[quarter].sum()
        quarterly_totals.append(total)
    
    fig_eac.add_trace(
        go.Scatter(
            x=quarters,
            y=quarterly_totals,
            mode='lines+markers',
            name='Total EAC Trade',
            line=dict(color='#F18F01', width=3),
            marker=dict(size=6)
        ),
        row=2, col=2
    )
    
    # Update layout
    fig_eac.update_layout(
        height=1000,
        title_text="East African Community (EAC) Trade Analysis",
        title_x=0.5,
        showlegend=True
    )
    
    # Update axes
    fig_eac.update_xaxes(tickangle=45, row=2, col=2)
    
    fig_eac.show()
    
    print("✅ EAC trade analysis visualization completed")
else:
    print("❌ EAC data not available for visualization")

✅ EAC trade analysis visualization completed


In [11]:
# Analyze regional blocks and continental trade
regional_sheets = ['Regional blocks', 'Trade by continents']

for sheet_name in regional_sheets:
    if sheet_name in cleaned_data:
        df = cleaned_data[sheet_name]
        
        print(f"\n🌍 {sheet_name.upper()} Analysis")
        print("=" * (len(sheet_name) + 20))
        
        # Display basic information
        print(f"📊 Data shape: {df.shape[0]} rows × {df.shape[1]} columns")
        print(f"📋 Columns: {list(df.columns)}")
        
        # Show sample data
        print(f"\n📋 Sample Data:")
        display(df.head())
        
        # Analyze if it's continental data
        if 'continent' in sheet_name.lower():
            # Extract export data by continent
            continent_exports = {}
            for _, row in df.iterrows():
                if isinstance(row.get('Flow'), str) and row.get('Flow') == 'Exports':
                    continent = row.get('Partner')
                    value = row.get('2024Q4', 0)
                    if pd.notna(value) and continent != 'WORLD':
                        continent_exports[continent] = float(value)
            
            if continent_exports:
                print(f"\n🌍 Continental Export Analysis (Q4 2024):")
                for continent, value in sorted(continent_exports.items(), key=lambda x: x[1], reverse=True):
                    print(f"   {continent}: ${value:.2f}M")
                
                # Find top continent
                top_continent = max(continent_exports.items(), key=lambda x: x[1])
                print(f"\n🏆 Top Export Destination: {top_continent[0]} - ${top_continent[1]:.2f}M")
                
                # Calculate diversity
                print(f"📊 Geographic Diversity: {len(continent_exports)} continents")
        
        print(f"\n📋 Complete {sheet_name} Data:")
        display(df)
        
    else:
        print(f"❌ {sheet_name} not found in cleaned data")


🌍 REGIONAL BLOCKS Analysis
📊 Data shape: 29 rows × 14 columns
📋 Columns: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13']

📋 Sample Data:


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Trade in Goods of Rwanda with selected regiona...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Partner,Flow \ Period,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2,2024Q3,2024Q4
3,CEPGL,Export,35.453783,36.816685,41.354027,44.760689,46.584752,41.899511,56.303221,52.473826,52.190233,49.710889,65.194014,86.555938
4,NaN,Import,10.440299,11.889871,8.350646,8.115722,4.542269,6.279899,8.915258,12.066122,14.09492,11.220648,20.798657,27.148592
5,NaN,Re-export,144.074346,154.124102,186.811749,180.860722,147.703243,145.404962,162.007151,153.458057,164.254777,158.049767,176.222145,167.58884



📋 Complete Regional blocks Data:


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Trade in Goods of Rwanda with selected regiona...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Partner,Flow \ Period,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2,2024Q3,2024Q4
3,CEPGL,Export,35.453783,36.816685,41.354027,44.760689,46.584752,41.899511,56.303221,52.473826,52.190233,49.710889,65.194014,86.555938
4,NaN,Import,10.440299,11.889871,8.350646,8.115722,4.542269,6.279899,8.915258,12.066122,14.09492,11.220648,20.798657,27.148592
5,NaN,Re-export,144.074346,154.124102,186.811749,180.860722,147.703243,145.404962,162.007151,153.458057,164.254777,158.049767,176.222145,167.58884
6,NaN,Total Trade,189.968429,202.830657,236.516422,233.737133,198.830265,193.584372,227.22563,217.998004,230.53993,218.981304,262.214816,281.293369
7,COMESA,Export,50.060488,54.378775,68.105831,66.153482,67.797563,58.378172,72.095417,61.817258,60.754547,60.360341,79.389357,102.062542
8,NaN,Import,141.197418,217.268365,217.983642,207.641884,237.095777,195.912019,227.121321,186.929624,201.582195,338.422149,331.194472,344.952128
9,NaN,Re-export,146.56179,157.838326,194.678938,186.294066,152.931638,148.747893,165.590382,155.612661,169.340541,160.654836,180.518569,172.714011
10,NaN,Total Trade,337.819695,429.485466,480.768411,460.089432,457.824978,403.038085,464.807121,404.359543,431.677284,559.437326,591.102398,619.728682



🌍 TRADE BY CONTINENTS Analysis
📊 Data shape: 21 rows × 26 columns
📋 Columns: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Trade in Goods of Rwanda by Continents', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25']

📋 Sample Data:


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Trade in Goods of Rwanda by Continents,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
1,NaN,NaN,NaN,VALUES IN US$,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SHARE IN %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Flow,Partner \ Period,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2,2024Q3,2024Q4,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2,2024Q3,2024Q4
3,Exports,WORLD,293.578848,331.552551,342.563538,354.838691,402.144688,484.739346,388.112971,399.108401,431.612022,537.637796,667.004575,677.446205,100,100,100,100,100,100,100,100,100,100,100,100
4,NaN,AFRICA,75.48,65.215975,86.26,81.805987,91.749864,73.311712,83.861104,71.635173,72.31,69.6,87.32,112.92,25.710299,19.669876,25.18073,23.054416,22.815138,15.123945,21.607395,17.948801,16.753472,12.945518,13.091364,16.668482
5,NaN,AMERICA,3.820708,4.1,5.38,11.61,3.12,2.59,7.776345,9.276707,4.76,7.85,9.36,10.6,1.301425,1.236606,1.570512,3.271909,0.77584,0.534308,2.003629,2.324358,1.102842,1.460091,1.403289,1.5647



📋 Complete Trade by continents Data:


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Trade in Goods of Rwanda by Continents,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
1,NaN,NaN,NaN,VALUES IN US$,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SHARE IN %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Flow,Partner \ Period,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2,2024Q3,2024Q4,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2,2024Q3,2024Q4
3,Exports,WORLD,293.578848,331.552551,342.563538,354.838691,402.144688,484.739346,388.112971,399.108401,431.612022,537.637796,667.004575,677.446205,100,100,100,100,100,100,100,100,100,100,100,100
4,NaN,AFRICA,75.48,65.215975,86.26,81.805987,91.749864,73.311712,83.861104,71.635173,72.31,69.6,87.32,112.92,25.710299,19.669876,25.18073,23.054416,22.815138,15.123945,21.607395,17.948801,16.753472,12.945518,13.091364,16.668482
5,NaN,AMERICA,3.820708,4.1,5.38,11.61,3.12,2.59,7.776345,9.276707,4.76,7.85,9.36,10.6,1.301425,1.236606,1.570512,3.271909,0.77584,0.534308,2.003629,2.324358,1.102842,1.460091,1.403289,1.5647
6,NaN,ASIA,185.156273,240.850321,210.12,208.208394,269.04,373.966047,263.52,270.73,319.98,418.283575,509.674629,499.034518,63.068669,72.643181,61.337526,58.676914,66.901294,77.147863,67.897756,67.833701,74.136026,77.800255,76.412464,73.66408
7,NaN,EUROPE,29.087444,21.332333,39.75,52.373654,37.874867,34.832891,32.351697,46.756108,34.562468,41.834739,59.923903,53.962076,9.907881,6.434073,11.603687,14.759849,9.418219,7.185901,8.335639,11.71514,8.007763,7.781212,8.984032,7.965515
8,NaN,OCEANIA,0.034949,0.055341,1.053881,0.83983,0.363539,0.039508,0.607062,0.711749,0,0.06677,0.725716,0.924838,0.011905,0.016691,0.307645,0.236679,0.0904,0.00815,0.156414,0.178335,0,0.012419,0.108802,0.136518
9,Imports,WORLD,1034.539426,1348.025947,1481.224343,1281.207428,1476.509948,1571.094626,1581.814476,1486.934303,1410.520447,1568.972564,1751.568934,1629.389784,100,100,100,100,100,100,100,100,100,100,100,100
10,NaN,AFRICA,327.61,452.63,487.84,386.62,511.625099,584.693086,529.306604,549.9,543.75,645.28,861.6,778.55,31.667232,33.577247,32.934916,30.176222,34.650975,37.215651,33.46199,36.982132,38.549601,41.127552,49.190185,47.781692


In [ ]:
# 3. Country-Specific Trade Analysis

This section analyzes Rwanda's trade relationships with individual countries, focusing on top export destinations, import sources, and re-export partners.

In [12]:
# Analyze top export countries
if 'ExportCountry' in cleaned_data:
    export_countries = cleaned_data['ExportCountry']
    
    print("🚀 Export Countries Analysis")
    print("=" * 35)
    print(f"📊 Countries analyzed: {len(export_countries)}")
    
    # Find top export destinations
    top_exports = export_countries.nlargest(10, 'q4_2024')
    
    print(f"\n🏆 Top 10 Export Destinations (Q4 2024):")
    for i, (_, country) in enumerate(top_exports.iterrows(), 1):
        print(f"   {i}. {country['country']}: ${country['q4_2024']:.2f}M ({country['share_q4']:.2f}%)")
    
    # Calculate concentration
    total_exports = export_countries['q4_2024'].sum()
    top_5_share = top_exports.head(5)['q4_2024'].sum() / total_exports * 100
    
    print(f"\n📈 Export Concentration:")
    print(f"   Top 5 countries: {top_5_share:.2f}% of total exports")
    print(f"   Export diversity: {len(export_countries)} countries")
    
    # Calculate growth analysis
    export_countries['yoy_growth'] = ((export_countries['q4_2024'] - export_countries['q4_2023']) / export_countries['q4_2023']) * 100
    avg_growth = export_countries['yoy_growth'].mean()
    
    print(f"\n📈 Average YoY Growth: {avg_growth:.2f}%")
    
    # Find fastest growing markets
    growing_markets = export_countries[export_countries['yoy_growth'] > 0].nlargest(5, 'yoy_growth')
    print(f"\n🚀 Fastest Growing Markets:")
    for _, country in growing_markets.iterrows():
        print(f"   {country['country']}: {country['yoy_growth']:.2f}% growth")
    
    print(f"\n📋 Top Export Countries Data:")
    display(top_exports[['country', 'q4_2024', 'share_q4', 'growth_qoq', 'growth_yoy']])
    
else:
    print("❌ ExportCountry sheet not found in cleaned data")

🚀 Export Countries Analysis
📊 Countries analyzed: 20

🏆 Top 10 Export Destinations (Q4 2024):
   1. China: $20.43M (3.02%)
   2. Luxembourg: $14.10M (2.08%)
   3. United Kingdom: $9.31M (1.37%)
   4. United States: $8.97M (1.32%)
   5. Uganda: $7.50M (1.11%)
   6. India: $6.18M (0.91%)
   7. Hong Kong: $5.89M (0.87%)
   8. Netherlands: $5.76M (0.85%)
   9. Italy: $5.72M (0.84%)
   10. Belgium: $5.44M (0.80%)

📈 Export Concentration:
   Top 5 countries: 51.60% of total exports
   Export diversity: 20 countries

📈 Average YoY Growth: 402858.56%

🚀 Fastest Growing Markets:
   Ethiopia: 7249960.82% growth
   Thailand: 939.90% growth
   Belgium: 145.20% growth
   Italy: 117.04% growth
   Singapore: 90.20% growth

📋 Top Export Countries Data:


,country,q4_2024,share_q4,growth_qoq,growth_yoy
0,China,20.427312,3.015341,-0.105413,0.069333
1,Luxembourg,14.104303,2.081981,0.024366,0.760957
2,United Kingdom,9.310813,1.374399,-0.146387,0.171064
3,United States,8.968663,1.323893,0.098773,0.021665
4,Uganda,7.503907,1.107676,0.656391,0.879983
5,India,6.180915,0.912385,1.351602,0.609136
6,Hong Kong,5.888581,0.869232,-0.309247,-0.402163
7,Netherlands,5.757677,0.849909,-0.149403,0.737200
8,Italy,5.722292,0.844686,3.760777,1.170395
9,Belgium,5.444409,0.803667,-0.022470,1.451966


In [13]:
# Analyze top import countries
if 'ImportCountry' in cleaned_data:
    import_countries = cleaned_data['ImportCountry']
    
    print("📦 Import Countries Analysis")
    print("=" * 35)
    print(f"📊 Countries analyzed: {len(import_countries)}")
    
    # Find top import sources
    top_imports = import_countries.nlargest(10, 'q4_2024')
    
    print(f"\n🏆 Top 10 Import Sources (Q4 2024):")
    for i, (_, country) in enumerate(top_imports.iterrows(), 1):
        print(f"   {i}. {country['country']}: ${country['q4_2024']:.2f}M ({country['share_q4']:.2f}%)")
    
    # Calculate concentration
    total_imports = import_countries['q4_2024'].sum()
    top_5_import_share = top_imports.head(5)['q4_2024'].sum() / total_imports * 100
    
    print(f"\n📈 Import Concentration:")
    print(f"   Top 5 countries: {top_5_import_share:.2f}% of total imports")
    print(f"   Import diversity: {len(import_countries)} countries")
    
    # Calculate growth analysis
    import_countries['yoy_growth'] = ((import_countries['q4_2024'] - import_countries['q4_2023']) / import_countries['q4_2023']) * 100
    avg_import_growth = import_countries['yoy_growth'].mean()
    
    print(f"\n📈 Average YoY Growth: {avg_import_growth:.2f}%")
    
    print(f"\n📋 Top Import Countries Data:")
    display(top_imports[['country', 'q4_2024', 'share_q4', 'growth_qoq', 'growth_yoy']])
    
else:
    print("❌ ImportCountry sheet not found in cleaned data")

📦 Import Countries Analysis
📊 Countries analyzed: 18

🏆 Top 10 Import Sources (Q4 2024):
   1. Kenya: $211.22M (12.96%)
   2. India: $101.83M (6.25%)
   3. United Arab Emirates: $96.64M (5.93%)
   4. Uganda: $75.13M (4.61%)
   5. South Africa: $55.75M (3.42%)
   6. Burkina Faso: $50.32M (3.09%)
   7. Malaysia: $42.42M (2.60%)
   8. Japan: $28.01M (1.72%)
   9. Congo, The Democratic Republic Of: $26.50M (1.63%)
   10. Germany: $21.65M (1.33%)

📈 Import Concentration:
   Top 5 countries: 64.44% of total imports
   Import diversity: 18 countries

📈 Average YoY Growth: 102448.09%

📋 Top Import Countries Data:


,country,q4_2024,share_q4,growth_qoq,growth_yoy
0,Kenya,211.221117,12.963204,-0.018532,1.990130
1,India,101.828859,6.249509,-0.129835,-0.117293
2,United Arab Emirates,96.635552,5.930782,0.101164,0.117135
3,Uganda,75.134446,4.611202,0.184540,0.056476
4,South Africa,55.745400,3.421244,2.751732,-0.292518
5,Burkina Faso,50.318168,3.088160,-0.363121,18438.173427
6,Malaysia,42.418297,2.603324,0.184671,1.058928
7,Japan,28.014673,1.719335,-0.042078,0.084548
8,"Congo, The Democratic Republic Of",26.500727,1.626420,0.285553,1.355789
9,Germany,21.650714,1.328762,0.083865,0.380987


In [14]:
# Analyze re-export countries
if 'ReexportsCountry' in cleaned_data:
    reexport_countries = cleaned_data['ReexportsCountry']
    
    print("🔄 Re-export Countries Analysis")
    print("=" * 40)
    print(f"📊 Countries analyzed: {len(reexport_countries)}")
    
    # Find top re-export destinations
    top_reexports = reexport_countries.nlargest(10, 'q4_2024')
    
    print(f"\n🏆 Top 10 Re-export Destinations (Q4 2024):")
    for i, (_, country) in enumerate(top_reexports.iterrows(), 1):
        print(f"   {i}. {country['country']}: ${country['q4_2024']:.2f}M ({country['share_q4']:.2f}%)")
    
    # Calculate concentration
    total_reexports = reexport_countries['q4_2024'].sum()
    top_5_reexport_share = top_reexports.head(5)['q4_2024'].sum() / total_reexports * 100
    
    print(f"\n📈 Re-export Concentration:")
    print(f"   Top 5 countries: {top_5_reexport_share:.2f}% of total re-exports")
    print(f"   Re-export diversity: {len(reexport_countries)} countries")
    
    print(f"\n📋 Top Re-export Countries Data:")
    display(top_reexports[['country', 'q4_2024', 'share_q4', 'growth_qoq', 'growth_yoy']])
    
else:
    print("❌ ReexportsCountry sheet not found in cleaned data")

🔄 Re-export Countries Analysis
📊 Countries analyzed: 18

🏆 Top 10 Re-export Destinations (Q4 2024):
   1. United Arab Emirates: $1.97M (1.11%)
   2. Burundi: $1.26M (0.71%)
   3. Uganda: $1.12M (0.63%)
   4. United Kingdom: $0.93M (0.52%)
   5. Zambia: $0.90M (0.51%)
   6. South Sudan: $0.62M (0.35%)
   7. Kenya: $0.30M (0.17%)
   8. Belgium: $0.26M (0.15%)
   9. Turkey: $0.19M (0.11%)
   10. Greece: $0.16M (0.09%)

📈 Re-export Concentration:
   Top 5 countries: 76.29% of total re-exports
   Re-export diversity: 18 countries

📋 Top Re-export Countries Data:


,country,q4_2024,share_q4,growth_qoq,growth_yoy
0,United Arab Emirates,1.973311,1.113059,-0.005690,0.506704
1,Burundi,1.264607,0.713310,-0.225839,-0.638322
2,Uganda,1.120528,0.632041,0.403346,0.452592
3,United Kingdom,0.928749,0.523867,92.690722,0.278946
4,Zambia,0.896650,0.505762,2.231281,0.000000
5,South Sudan,0.616656,0.347829,-0.181588,1.303105
6,Kenya,0.297532,0.167825,0.850266,2.198654
7,Belgium,0.258548,0.145836,0.652876,2.129671
8,Turkey,0.189856,0.107089,-0.226625,-0.059298
9,Greece,0.159876,0.090179,0.487602,-0.063924


In [15]:
# Create comprehensive country analysis visualization - FIXED subplot issues
if all(sheet in cleaned_data for sheet in ['ExportCountry', 'ImportCountry', 'ReexportsCountry']):
    export_df = cleaned_data['ExportCountry']
    import_df = cleaned_data['ImportCountry']
    reexport_df = cleaned_data['ReexportsCountry']
    
    # Create comprehensive country analysis - FIXED subplot specs
    fig_countries = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Top Export Destinations (Q4 2024)',
            'Export Market Share Distribution',
            'Top Import Sources (Q4 2024)',
            'Import Source Distribution',
            'Top Re-export Destinations',
            'Trade Partner Comparison'
        ),
        specs=[[{"type": "bar"}, {"type": "domain"}],
               [{"type": "bar"}, {"type": "domain"}],
               [{"type": "bar"}, {"secondary_y": False}]]
    )
    
    # 1. Top Export Destinations
    top_exports = export_df.nlargest(10, 'q4_2024')
    fig_countries.add_trace(
        go.Bar(
            x=top_exports['q4_2024'],
            y=top_exports['country'],
            orientation='h',
            name='Export Volume',
            marker_color='#2E86AB'
        ),
        row=1, col=1
    )
    
    # 2. Export Market Share - FIXED: Pie chart in domain subplot
    fig_countries.add_trace(
        go.Pie(
            labels=top_exports['country'],
            values=top_exports['share_q4'],
            name='Export Share',
            textinfo='label+percent',
            textposition='inside'
        ),
        row=1, col=2
    )
    
    # 3. Top Import Sources
    top_imports = import_df.nlargest(10, 'q4_2024')
    fig_countries.add_trace(
        go.Bar(
            x=top_imports['q4_2024'],
            y=top_imports['country'],
            orientation='h',
            name='Import Volume',
            marker_color='#A23B72'
        ),
        row=2, col=1
    )
    
    # 4. Import Source Distribution - FIXED: Pie chart in domain subplot
    fig_countries.add_trace(
        go.Pie(
            labels=top_imports['country'],
            values=top_imports['share_q4'],
            name='Import Share',
            textinfo='label+percent',
            textposition='inside'
        ),
        row=2, col=2
    )
    
    # 5. Top Re-export Destinations
    top_reexports = reexport_df.nlargest(10, 'q4_2024')
    fig_countries.add_trace(
        go.Bar(
            x=top_reexports['q4_2024'],
            y=top_reexports['country'],
            orientation='h',
            name='Re-export Volume',
            marker_color='#F18F01'
        ),
        row=3, col=1
    )
    
    # 6. Trade Partner Comparison
    # Compare top partners across export, import, and re-export
    top_export_partner = top_exports.iloc[0]['country']
    top_import_partner = top_imports.iloc[0]['country']
    top_reexport_partner = top_reexports.iloc[0]['country']
    
    # Get values for comparison
    export_val = top_exports.iloc[0]['q4_2024']
    import_val = top_imports.iloc[0]['q4_2024']
    reexport_val = top_reexports.iloc[0]['q4_2024']
    
    fig_countries.add_trace(
        go.Bar(
            x=['Exports', 'Imports', 'Re-exports'],
            y=[export_val, import_val, reexport_val],
            name='Top Partner Comparison',
            marker_color=['#2E86AB', '#A23B72', '#F18F01']
        ),
        row=3, col=2
    )
    
    # Update layout
    fig_countries.update_layout(
        height=1200,
        title_text="Rwanda Country-Specific Trade Analysis (Q4 2024)",
        title_x=0.5,
        showlegend=True
    )
    
    fig_countries.show()
    
    print("✅ Country analysis visualization completed")
else:
    print("❌ Country data not available for visualization")

✅ Country analysis visualization completed


In [ ]:
# 4. Commodity Analysis

This section analyzes Rwanda's trade by commodity type using the Standard International Trade Classification (SITC) system, providing insights into product-specific trade patterns.

In [16]:
# Analyze export commodities
if 'ExportsCommodity' in cleaned_data:
    export_commodities = cleaned_data['ExportsCommodity']
    
    print("📦 Export Commodities Analysis")
    print("=" * 40)
    print(f"📊 Commodities analyzed: {len(export_commodities)}")
    
    # Find top export commodities
    top_export_commodities = export_commodities.nlargest(15, 'q4_2024')
    
    print(f"\n🏆 Top 15 Export Commodities (Q4 2024):")
    for i, (_, commodity) in enumerate(top_export_commodities.iterrows(), 1):
        sitc_section = commodity['sitc_section']
        description = commodity['description']
        value = commodity['q4_2024']
        share = commodity['share_q4']
        print(f"   {i}. SITC {sitc_section} - {description}: ${value:.2f}M ({share:.2f}%)")
    
    # Calculate commodity concentration
    total_exports = export_commodities['q4_2024'].sum()
    top_5_commodity_share = top_export_commodities.head(5)['q4_2024'].sum() / total_exports * 100
    
    print(f"\n📈 Export Commodity Concentration:")
    print(f"   Top 5 commodities: {top_5_commodity_share:.2f}% of total exports")
    print(f"   Export commodity diversity: {len(export_commodities)} commodities")
    
    # Analyze by SITC sections
    sitc_analysis = export_commodities.groupby('sitc_section').agg({
        'q4_2024': 'sum',
        'description': 'count'
    }).sort_values('q4_2024', ascending=False)
    
    print(f"\n🏭 Export Analysis by SITC Section:")
    for sitc, row in sitc_analysis.iterrows():
        section_name = SITC_SECTIONS.get(sitc, f'SITC Section {sitc}')
        value = row['q4_2024']
        count = row['description']
        share = (value / total_exports) * 100
        print(f"   SITC {sitc} ({section_name}): ${value:.2f}M ({share:.2f}%) - {count} commodities")
    
    print(f"\n📋 Top Export Commodities Data:")
    display(top_export_commodities[['sitc_section', 'description', 'q4_2024', 'share_q4', 'growth_qoq', 'growth_yoy']])
    
else:
    print("❌ ExportsCommodity sheet not found in cleaned data")

📦 Export Commodities Analysis
📊 Commodities analyzed: 10

🏆 Top 15 Export Commodities (Q4 2024):
   1. SITC 9 - Other commodities & transactions, n.e.s: $428.15M (63.20%)
   2. SITC 0 - Food and live animals: $101.12M (14.93%)
   3. SITC 2 - Crude materials, inedible, except fuels : $58.79M (8.68%)
   4. SITC 6 - Manufactured goods classified chiefly by material: $34.87M (5.15%)
   5. SITC 4 - Animals and vegetable oils, fats & waxes: $23.40M (3.45%)
   6. SITC 7 - Machinery and transport equipment: $16.87M (2.49%)
   7. SITC 8 - Miscellaneous manufactured articles: $8.32M (1.23%)
   8. SITC 5 - Chemicals & related products, n.e.s.: $4.88M (0.72%)
   9. SITC 3 - Mineral fuels, lubricants and related materials: $0.94M (0.14%)
   10. SITC 1 - Beverages and tobacco: $0.10M (0.01%)

📈 Export Commodity Concentration:
   Top 5 commodities: 95.41% of total exports
   Export commodity diversity: 10 commodities

🏭 Export Analysis by SITC Section:
   SITC 9 (Commodities and transactions not clas

,sitc_section,description,q4_2024,share_q4,growth_qoq,growth_yoy
9,9,"Other commodities & transactions, n.e.s",428.148348,63.200346,-0.015054,1.113087
0,0,Food and live animals,101.124930,14.927374,0.019745,-0.025881
2,2,"Crude materials, inedible, except fuels",58.789237,8.678067,-0.134262,0.148520
6,6,Manufactured goods classified chiefly by material,34.872048,5.147575,0.134955,0.415713
4,4,"Animals and vegetable oils, fats & waxes",23.396384,3.453615,0.943355,20.002958
7,7,Machinery and transport equipment,16.870762,2.490347,1.385293,4.945666
8,8,Miscellaneous manufactured articles,8.323205,1.228615,-0.120247,0.310390
5,5,"Chemicals & related products, n.e.s.",4.877972,0.720053,-0.000279,0.081264
3,3,"Mineral fuels, lubricants and related materials",0.940699,0.138860,-0.043038,-0.451448
1,1,Beverages and tobacco,0.101611,0.014999,0.341868,-0.691432


In [17]:
# Analyze import commodities
if 'ImportsCommodity' in cleaned_data:
    import_commodities = cleaned_data['ImportsCommodity']
    
    print("📦 Import Commodities Analysis")
    print("=" * 40)
    print(f"📊 Commodities analyzed: {len(import_commodities)}")
    
    # Find top import commodities
    top_import_commodities = import_commodities.nlargest(15, 'q4_2024')
    
    print(f"\n🏆 Top 15 Import Commodities (Q4 2024):")
    for i, (_, commodity) in enumerate(top_import_commodities.iterrows(), 1):
        sitc_section = commodity['sitc_section']
        description = commodity['description']
        value = commodity['q4_2024']
        share = commodity['share_q4']
        print(f"   {i}. SITC {sitc_section} - {description}: ${value:.2f}M ({share:.2f}%)")
    
    # Calculate commodity concentration
    total_imports = import_commodities['q4_2024'].sum()
    top_5_import_commodity_share = top_import_commodities.head(5)['q4_2024'].sum() / total_imports * 100
    
    print(f"\n📈 Import Commodity Concentration:")
    print(f"   Top 5 commodities: {top_5_import_commodity_share:.2f}% of total imports")
    print(f"   Import commodity diversity: {len(import_commodities)} commodities")
    
    # Analyze by SITC sections
    import_sitc_analysis = import_commodities.groupby('sitc_section').agg({
        'q4_2024': 'sum',
        'description': 'count'
    }).sort_values('q4_2024', ascending=False)
    
    print(f"\n🏭 Import Analysis by SITC Section:")
    for sitc, row in import_sitc_analysis.iterrows():
        section_name = SITC_SECTIONS.get(sitc, f'SITC Section {sitc}')
        value = row['q4_2024']
        count = row['description']
        share = (value / total_imports) * 100
        print(f"   SITC {sitc} ({section_name}): ${value:.2f}M ({share:.2f}%) - {count} commodities")
    
    print(f"\n📋 Top Import Commodities Data:")
    display(top_import_commodities[['sitc_section', 'description', 'q4_2024', 'share_q4', 'growth_qoq', 'growth_yoy']])
    
else:
    print("❌ ImportsCommodity sheet not found in cleaned data")

📦 Import Commodities Analysis
📊 Commodities analyzed: 10

🏆 Top 15 Import Commodities (Q4 2024):
   1. SITC 9 - Other commodities & transactions, n.e.s: $396.16M (24.31%)
   2. SITC 7 - Machinery and transport equipment: $238.86M (14.66%)
   3. SITC 0 - Food and live animals: $234.57M (14.40%)
   4. SITC 6 - Manufactured goods classified chiefly by material: $215.13M (13.20%)
   5. SITC 3 - Mineral fuels, lubricants and related materials: $190.53M (11.69%)
   6. SITC 5 - Chemicals & related products, n.e.s.: $135.39M (8.31%)
   7. SITC 8 - Miscellaneous manufactured articles: $92.66M (5.69%)
   8. SITC 4 - Animals and vegetable oils, fats & waxes: $55.92M (3.43%)
   9. SITC 2 - Crude materials, inedible, except fuels : $40.66M (2.50%)
   10. SITC 1 - Beverages and tobacco: $27.41M (1.68%)

📈 Import Commodity Concentration:
   Top 5 commodities: 78.37% of total imports
   Import commodity diversity: 10 commodities

🏭 Import Analysis by SITC Section:
   SITC 9 (Commodities and transactio

,sitc_section,description,q4_2024,share_q4,growth_qoq,growth_yoy
9,9,"Other commodities & transactions, n.e.s",396.162410,24.313545,-0.173146,0.958728
7,7,Machinery and transport equipment,238.860870,14.659529,-0.059091,-0.076427
0,0,Food and live animals,234.566433,14.395968,-0.012503,-0.192050
6,6,Manufactured goods classified chiefly by material,215.126416,13.202882,-0.015726,-0.008045
3,3,"Mineral fuels, lubricants and related materials",190.526784,11.693137,-0.057187,0.090691
5,5,"Chemicals & related products, n.e.s.",135.392143,8.309377,-0.078533,0.037716
8,8,Miscellaneous manufactured articles,92.662318,5.686934,0.061985,0.060282
4,4,"Animals and vegetable oils, fats & waxes",55.924706,3.432248,0.049649,-0.023461
2,2,"Crude materials, inedible, except fuels",40.664178,2.495669,-0.082607,0.037373
1,1,Beverages and tobacco,27.409199,1.682176,0.017155,0.025434


In [18]:
# Analyze re-export commodities
if 'ReexportsCommodity' in cleaned_data:
    reexport_commodities = cleaned_data['ReexportsCommodity']
    
    print("🔄 Re-export Commodities Analysis")
    print("=" * 45)
    print(f"📊 Commodities analyzed: {len(reexport_commodities)}")
    
    # Find top re-export commodities
    top_reexport_commodities = reexport_commodities.nlargest(15, 'q4_2024')
    
    print(f"\n🏆 Top 15 Re-export Commodities (Q4 2024):")
    for i, (_, commodity) in enumerate(top_reexport_commodities.iterrows(), 1):
        sitc_section = commodity['sitc_section']
        description = commodity['description']
        value = commodity['q4_2024']
        share = commodity['share_q4']
        print(f"   {i}. SITC {sitc_section} - {description}: ${value:.2f}M ({share:.2f}%)")
    
    # Calculate commodity concentration
    total_reexports = reexport_commodities['q4_2024'].sum()
    top_5_reexport_commodity_share = top_reexport_commodities.head(5)['q4_2024'].sum() / total_reexports * 100
    
    print(f"\n📈 Re-export Commodity Concentration:")
    print(f"   Top 5 commodities: {top_5_reexport_commodity_share:.2f}% of total re-exports")
    print(f"   Re-export commodity diversity: {len(reexport_commodities)} commodities")
    
    print(f"\n📋 Top Re-export Commodities Data:")
    display(top_reexport_commodities[['sitc_section', 'description', 'q4_2024', 'share_q4', 'growth_qoq', 'growth_yoy']])
    
else:
    print("❌ ReexportsCommodity sheet not found in cleaned data")

🔄 Re-export Commodities Analysis
📊 Commodities analyzed: 9

🏆 Top 15 Re-export Commodities (Q4 2024):
   1. SITC 3 - Mineral fuels, lubricants and related materials: $48.01M (27.08%)
   2. SITC 6 - Manufactured goods classified chiefly by material: $20.40M (11.51%)
   3. SITC 1 - Beverages and tobacco: $12.25M (6.91%)
   4. SITC 5 - Chemicals & related products, n.e.s.: $11.62M (6.55%)
   5. SITC 8 - Miscellaneous manufactured articles: $10.18M (5.74%)
   6. SITC 7 - Machinery and transport equipment: $9.99M (5.63%)
   7. SITC 4 - Animals and vegetable oils, fats & waxes: $8.39M (4.73%)
   8. SITC 2 - Crude materials, inedible, except fuels : $4.87M (2.75%)
   9. SITC 9 - Other commodities & transactions, n.e.s: $0.00M (0.00%)

📈 Re-export Commodity Concentration:
   Top 5 commodities: 81.51% of total re-exports
   Re-export commodity diversity: 9 commodities

📋 Top Re-export Commodities Data:


,sitc_section,description,q4_2024,share_q4,growth_qoq,growth_yoy
2,3,"Mineral fuels, lubricants and related materials",48.010717,27.080774,-0.106630,0.239402
5,6,Manufactured goods classified chiefly by material,20.404714,11.509419,-0.071682,0.421531
0,1,Beverages and tobacco,12.247577,6.908330,0.171453,0.449222
4,5,"Chemicals & related products, n.e.s.",11.616736,6.552499,-0.036041,0.484193
7,8,Miscellaneous manufactured articles,10.184697,5.744748,0.039106,0.719214
6,7,Machinery and transport equipment,9.987037,5.633256,-0.025837,0.270894
3,4,"Animals and vegetable oils, fats & waxes",8.386770,4.730615,-0.067458,-0.467984
1,2,"Crude materials, inedible, except fuels",4.868388,2.746048,-0.016980,0.081586
8,9,"Other commodities & transactions, n.e.s",0.000000,0.000000,0.000000,0.000000


In [19]:
# Create comprehensive commodity analysis visualization - FIXED subplot issues
if all(sheet in cleaned_data for sheet in ['ExportsCommodity', 'ImportsCommodity', 'ReexportsCommodity']):
    export_comm = cleaned_data['ExportsCommodity']
    import_comm = cleaned_data['ImportsCommodity']
    reexport_comm = cleaned_data['ReexportsCommodity']
    
    # Create comprehensive commodity analysis - FIXED subplot specs
    fig_commodities = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Top Export Commodities (Q4 2024)',
            'Export Commodities by SITC Section',
            'Top Import Commodities (Q4 2024)',
            'Import Commodities by SITC Section',
            'Top Re-export Commodities',
            'Commodity Trade Comparison'
        ),
        specs=[[{"type": "bar"}, {"type": "domain"}],
               [{"type": "bar"}, {"type": "domain"}],
               [{"type": "bar"}, {"secondary_y": False}]]
    )
    
    # 1. Top Export Commodities
    top_export_comm = export_comm.nlargest(10, 'q4_2024')
    fig_commodities.add_trace(
        go.Bar(
            x=top_export_comm['q4_2024'],
            y=top_export_comm['description'],
            orientation='h',
            name='Export Volume',
            marker_color='#2E86AB'
        ),
        row=1, col=1
    )
    
    # 2. Export Commodities by SITC Section - FIXED: Pie chart in domain subplot
    export_sitc = export_comm.groupby('sitc_section')['q4_2024'].sum().sort_values(ascending=True)
    fig_commodities.add_trace(
        go.Pie(
            labels=[SITC_SECTIONS.get(sitc, f'SITC {sitc}') for sitc in export_sitc.index],
            values=export_sitc.values,
            name='Export by SITC',
            textinfo='label+percent',
            textposition='inside'
        ),
        row=1, col=2
    )
    
    # 3. Top Import Commodities
    top_import_comm = import_comm.nlargest(10, 'q4_2024')
    fig_commodities.add_trace(
        go.Bar(
            x=top_import_comm['q4_2024'],
            y=top_import_comm['description'],
            orientation='h',
            name='Import Volume',
            marker_color='#A23B72'
        ),
        row=2, col=1
    )
    
    # 4. Import Commodities by SITC Section - FIXED: Pie chart in domain subplot
    import_sitc = import_comm.groupby('sitc_section')['q4_2024'].sum().sort_values(ascending=True)
    fig_commodities.add_trace(
        go.Pie(
            labels=[SITC_SECTIONS.get(sitc, f'SITC {sitc}') for sitc in import_sitc.index],
            values=import_sitc.values,
            name='Import by SITC',
            textinfo='label+percent',
            textposition='inside'
        ),
        row=2, col=2
    )
    
    # 5. Top Re-export Commodities
    top_reexport_comm = reexport_comm.nlargest(10, 'q4_2024')
    fig_commodities.add_trace(
        go.Bar(
            x=top_reexport_comm['q4_2024'],
            y=top_reexport_comm['description'],
            orientation='h',
            name='Re-export Volume',
            marker_color='#F18F01'
        ),
        row=3, col=1
    )
    
    # 6. Commodity Trade Comparison
    # Compare top commodities across export, import, and re-export
    top_export_comm_name = top_export_comm.iloc[0]['description']
    top_import_comm_name = top_import_comm.iloc[0]['description']
    top_reexport_comm_name = top_reexport_comm.iloc[0]['description']
    
    export_val = top_export_comm.iloc[0]['q4_2024']
    import_val = top_import_comm.iloc[0]['q4_2024']
    reexport_val = top_reexport_comm.iloc[0]['q4_2024']
    
    fig_commodities.add_trace(
        go.Bar(
            x=['Top Export', 'Top Import', 'Top Re-export'],
            y=[export_val, import_val, reexport_val],
            name='Top Commodity Comparison',
            marker_color=['#2E86AB', '#A23B72', '#F18F01']
        ),
        row=3, col=2
    )
    
    # Update layout
    fig_commodities.update_layout(
        height=1200,
        title_text="Rwanda Commodity Trade Analysis (Q4 2024)",
        title_x=0.5,
        showlegend=True
    )
    
    fig_commodities.show()
    
    print("✅ Commodity analysis visualization completed")
else:
    print("❌ Commodity data not available for visualization")

✅ Commodity analysis visualization completed


In [ ]:
# 5. Trend Analysis and Forecasting

This section provides advanced trend analysis and forecasting using statistical methods to predict future trade patterns.

In [20]:
# Perform trend analysis on overall trade data
if 'Graph Overall' in cleaned_data:
    overall_df = cleaned_data['Graph Overall']
    
    print("📈 Trend Analysis and Forecasting")
    print("=" * 45)
    
    # Calculate moving averages
    overall_df['export_ma_4'] = overall_df['exports'].rolling(window=4).mean()
    overall_df['import_ma_4'] = overall_df['imports'].rolling(window=4).mean()
    overall_df['trade_balance_ma_4'] = overall_df['trade_balance'].rolling(window=4).mean()
    
    # Calculate year-over-year growth
    overall_df['export_yoy'] = overall_df['exports'].pct_change(periods=4) * 100
    overall_df['import_yoy'] = overall_df['imports'].pct_change(periods=4) * 100
    
    print(f"\n📊 Moving Averages Analysis:")
    print(f"   Latest Export MA (4Q): ${overall_df['export_ma_4'].iloc[-1]:.2f}M")
    print(f"   Latest Import MA (4Q): ${overall_df['import_ma_4'].iloc[-1]:.2f}M")
    print(f"   Latest Trade Balance MA (4Q): ${overall_df['trade_balance_ma_4'].iloc[-1]:.2f}M")
    
    print(f"\n📈 Year-over-Year Growth:")
    print(f"   Latest Export YoY: {overall_df['export_yoy'].iloc[-1]:.2f}%")
    print(f"   Latest Import YoY: {overall_df['import_yoy'].iloc[-1]:.2f}%")
    
    # Calculate volatility
    export_volatility = overall_df['exports'].std() / overall_df['exports'].mean() * 100
    import_volatility = overall_df['imports'].std() / overall_df['imports'].mean() * 100
    
    print(f"\n📉 Volatility Analysis:")
    print(f"   Export Volatility: {export_volatility:.2f}%")
    print(f"   Import Volatility: {import_volatility:.2f}%")
    
    # Simple forecasting using linear regression
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import r2_score
    
    # Prepare data for forecasting
    X = np.array(range(len(overall_df))).reshape(-1, 1)
    y_export = overall_df['exports'].values
    y_import = overall_df['imports'].values
    
    # Fit linear regression models
    export_model = LinearRegression()
    import_model = LinearRegression()
    
    export_model.fit(X, y_export)
    import_model.fit(X, y_import)
    
    # Calculate R-squared scores
    export_r2 = r2_score(y_export, export_model.predict(X))
    import_r2 = r2_score(y_import, import_model.predict(X))
    
    print(f"\n🤖 Linear Regression Forecasting:")
    print(f"   Export Model R²: {export_r2:.4f}")
    print(f"   Import Model R²: {import_r2:.4f}")
    
    # Predict next quarter
    next_quarter = len(overall_df)
    export_forecast = export_model.predict([[next_quarter]])[0]
    import_forecast = import_model.predict([[next_quarter]])[0]
    
    print(f"\n🔮 Next Quarter Forecast:")
    print(f"   Predicted Exports: ${export_forecast:.2f}M")
    print(f"   Predicted Imports: ${import_forecast:.2f}M")
    print(f"   Predicted Trade Balance: ${export_forecast - import_forecast:.2f}M")
    
    # Display trend analysis data
    print(f"\n📋 Trend Analysis Data:")
    display(overall_df[['period', 'exports', 'imports', 'trade_balance', 'export_ma_4', 'import_ma_4', 'export_yoy', 'import_yoy']].tail())
    
else:
    print("❌ Graph Overall data not available for trend analysis")

📈 Trend Analysis and Forecasting

📊 Moving Averages Analysis:
   Latest Export MA (4Q): $222.12M
   Latest Import MA (4Q): $246.65M
   Latest Trade Balance MA (4Q): $279.20M

📈 Year-over-Year Growth:
   Latest Export YoY: -100.00%
   Latest Import YoY: -100.00%

📉 Volatility Analysis:
   Export Volatility: 206.07%
   Import Volatility: 224.42%

🤖 Linear Regression Forecasting:
   Export Model R²: 0.1670
   Import Model R²: 0.1605

🔮 Next Quarter Forecast:
   Predicted Exports: $-189.08M
   Predicted Imports: $-268.47M
   Predicted Trade Balance: $79.39M

📋 Trend Analysis Data:


,period,exports,imports,trade_balance,export_ma_4,import_ma_4,export_yoy,import_yoy
2,Re-Exports,150.655397,161.739440,156.252774,NaN,NaN,NaN,NaN
3,Total Trade,1478.773671,1841.317938,2034.907411,739.386836,920.658969,NaN,NaN
4,Trade Balance,-590.305180,-854.733955,-918.112486,518.415828,624.087342,-301.072108,-357.79743
5,*Major revisions include Domestic exports des...,0.000000,0.000000,0.000000,259.780972,287.080856,-100.000000,-100.00000
6,*All series revised from 2022Q1,0.000000,0.000000,0.000000,222.117123,246.645996,-100.000000,-100.00000


In [21]:
# Create trend analysis visualization
if 'Graph Overall' in cleaned_data:
    overall_df = cleaned_data['Graph Overall']
    
    # Create comprehensive trend analysis
    fig_trends = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Trade Volume Trends with Moving Averages',
            'Trade Balance Trend',
            'Year-over-Year Growth Rates',
            'Volatility Analysis',
            'Export vs Import Ratio',
            'Forecasting Model'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # 1. Trade Volume Trends with Moving Averages
    fig_trends.add_trace(
        go.Scatter(
            x=overall_df['period'], 
            y=overall_df['exports'], 
            mode='lines+markers', 
            name='Exports',
            line=dict(color='#2E86AB', width=2)
        ),
        row=1, col=1
    )
    
    fig_trends.add_trace(
        go.Scatter(
            x=overall_df['period'], 
            y=overall_df['export_ma_4'], 
            mode='lines', 
            name='Export MA (4Q)',
            line=dict(color='#2E86AB', width=4, dash='dash')
        ),
        row=1, col=1
    )
    
    fig_trends.add_trace(
        go.Scatter(
            x=overall_df['period'], 
            y=overall_df['imports'], 
            mode='lines+markers', 
            name='Imports',
            line=dict(color='#A23B72', width=2)
        ),
        row=1, col=1
    )
    
    fig_trends.add_trace(
        go.Scatter(
            x=overall_df['period'], 
            y=overall_df['import_ma_4'], 
            mode='lines', 
            name='Import MA (4Q)',
            line=dict(color='#A23B72', width=4, dash='dash')
        ),
        row=1, col=1
    )
    
    # 2. Trade Balance Trend
    fig_trends.add_trace(
        go.Scatter(
            x=overall_df['period'], 
            y=overall_df['trade_balance'], 
            mode='lines+markers', 
            name='Trade Balance',
            line=dict(color='#F18F01', width=3),
            fill='tozeroy',
            fillcolor='rgba(241, 143, 1, 0.3)'
        ),
        row=1, col=2
    )
    
    # 3. Year-over-Year Growth Rates
    fig_trends.add_trace(
        go.Scatter(
            x=overall_df['period'][4:], 
            y=overall_df['export_yoy'][4:], 
            mode='lines+markers', 
            name='Export YoY Growth %',
            line=dict(color='#2E86AB', width=2)
        ),
        row=2, col=1
    )
    
    fig_trends.add_trace(
        go.Scatter(
            x=overall_df['period'][4:], 
            y=overall_df['import_yoy'][4:], 
            mode='lines+markers', 
            name='Import YoY Growth %',
            line=dict(color='#A23B72', width=2)
        ),
        row=2, col=1
    )
    
    # 4. Volatility Analysis
    quarters = overall_df['period']
    export_vol = overall_df['exports'].rolling(window=4).std()
    import_vol = overall_df['imports'].rolling(window=4).std()
    
    fig_trends.add_trace(
        go.Scatter(
            x=quarters[3:], 
            y=export_vol[3:], 
            mode='lines', 
            name='Export Volatility',
            line=dict(color='#2E86AB', width=2)
        ),
        row=2, col=2
    )
    
    fig_trends.add_trace(
        go.Scatter(
            x=quarters[3:], 
            y=import_vol[3:], 
            mode='lines', 
            name='Import Volatility',
            line=dict(color='#A23B72', width=2)
        ),
        row=2, col=2
    )
    
    # 5. Export vs Import Ratio
    export_import_ratio = overall_df['exports'] / overall_df['imports'] * 100
    fig_trends.add_trace(
        go.Scatter(
            x=overall_df['period'], 
            y=export_import_ratio, 
            mode='lines+markers', 
            name='Export/Import Ratio %',
            line=dict(color='#C73E1D', width=3)
        ),
        row=3, col=1
    )
    
    # 6. Forecasting Model
    # Add actual data
    fig_trends.add_trace(
        go.Scatter(
            x=overall_df['period'], 
            y=overall_df['exports'], 
            mode='markers', 
            name='Actual Exports',
            marker=dict(color='#2E86AB', size=6)
        ),
        row=3, col=2
    )
    
    # Add trend line
    X_plot = np.array(range(len(overall_df)))
    y_plot = export_model.predict(X_plot.reshape(-1, 1))
    
    fig_trends.add_trace(
        go.Scatter(
            x=overall_df['period'], 
            y=y_plot, 
            mode='lines', 
            name=f'Trend Line (R²={export_r2:.3f})',
            line=dict(color='#2E86AB', width=3, dash='dash')
        ),
        row=3, col=2
    )
    
    # Add forecast point
    next_period = f"Q{((len(overall_df) % 4) + 1)} 2025"
    fig_trends.add_trace(
        go.Scatter(
            x=[next_period], 
            y=[export_forecast], 
            mode='markers', 
            name='Forecast',
            marker=dict(color='#F18F01', size=10, symbol='star')
        ),
        row=3, col=2
    )
    
    # Update layout
    fig_trends.update_layout(
        height=1200,
        title_text="Rwanda Trade Trend Analysis and Forecasting (2022-2024)",
        title_x=0.5,
        showlegend=True
    )
    
    # Update x-axis labels
    for i in range(1, 4):
        for j in range(1, 3):
            fig_trends.update_xaxes(tickangle=45, row=i, col=j)
    
    fig_trends.show()
    
    print("✅ Trend analysis visualization completed")
else:
    print("❌ Trend analysis data not available for visualization")

✅ Trend analysis visualization completed


In [ ]:
# 6. Key Insights and Recommendations

This section provides comprehensive insights derived from the analysis and actionable recommendations for trade policy and business strategy.

In [22]:
# Generate comprehensive insights and recommendations
def generate_insights_and_recommendations():
    """
    Generate comprehensive insights and recommendations based on the analysis
    """
    insights = []
    recommendations = []
    
    # Overall Trade Insights
    if 'Graph Overall' in cleaned_data:
        overall_df = cleaned_data['Graph Overall']
        latest_data = overall_df.iloc[-1]
        previous_data = overall_df.iloc[-2]
        
        # Trade balance analysis
        if latest_data['trade_balance'] < 0:
            deficit_ratio = abs(latest_data['trade_balance']) / latest_data['imports'] * 100
            insights.append({
                'category': 'Trade Balance',
                'type': 'warning',
                'insight': f"Rwanda maintains a significant trade deficit of ${latest_data['trade_balance']:.2f}M in Q4 2024",
                'details': f"This represents {deficit_ratio:.1f}% of total imports, indicating continued reliance on imported goods"
            })
        
        # Export growth analysis
        export_growth = ((latest_data['exports'] - previous_data['exports']) / previous_data['exports']) * 100
        if export_growth > 10:
            insights.append({
                'category': 'Export Performance',
                'type': 'success',
                'insight': f"Strong export growth of {export_growth:.1f}% quarter-over-quarter in Q4 2024",
                'details': f"Exports reached ${latest_data['exports']:.2f}M, showing robust performance"
            })
        
        # Import trends
        import_growth = ((latest_data['imports'] - previous_data['imports']) / previous_data['imports']) * 100
        if import_growth > 5:
            insights.append({
                'category': 'Import Trends',
                'type': 'info',
                'insight': f"Imports growing at {import_growth:.1f}% QoQ, indicating strong domestic demand",
                'details': f"Import value reached ${latest_data['imports']:.2f}M in Q4 2024"
            })
    
    # Regional Analysis Insights
    if 'EAC' in cleaned_data:
        eac_df = cleaned_data['EAC']
        total_eac = eac_df['q4_2024'].sum()
        eac_share = (total_eac / overall_df.iloc[-1]['exports']) * 100 if 'Graph Overall' in cleaned_data else 0
        
        if eac_share > 30:
            insights.append({
                'category': 'Regional Integration',
                'type': 'success',
                'insight': f"Strong regional integration with EAC accounting for {eac_share:.1f}% of exports",
                'details': f"EAC exports totaled ${total_eac:.2f}M in Q4 2024"
            })
    
    # Country Concentration Insights
    if 'ExportCountry' in cleaned_data:
        export_countries = cleaned_data['ExportCountry']
        top_5_export_share = export_countries.nlargest(5, 'q4_2024')['q4_2024'].sum() / export_countries['q4_2024'].sum() * 100
        
        if top_5_export_share > 80:
            insights.append({
                'category': 'Market Concentration',
                'type': 'warning',
                'insight': f"High export concentration with top 5 countries representing {top_5_export_share:.1f}% of exports",
                'details': "Consider market diversification strategies to reduce dependency"
            })
        
        # Find top export partner
        top_export_partner = export_countries.loc[export_countries['q4_2024'].idxmax()]
        if top_export_partner['share_q4'] > 50:
            insights.append({
                'category': 'Market Dependency',
                'type': 'warning',
                'insight': f"Heavy reliance on {top_export_partner['country']} for {top_export_partner['share_q4']:.1f}% of exports",
                'details': f"{top_export_partner['country']} imported ${top_export_partner['q4_2024']:.2f}M worth of Rwandan goods"
            })
    
    # Commodity Analysis Insights
    if 'ExportsCommodity' in cleaned_data:
        export_commodities = cleaned_data['ExportsCommodity']
        top_commodities = export_commodities.nlargest(5, 'q4_2024')
        top_commodity = top_commodities.iloc[0]
        
        insights.append({
            'category': 'Leading Export',
            'type': 'info',
            'insight': f"{top_commodity['description']} is Rwanda's leading export commodity",
            'details': f"Generated ${top_commodity['q4_2024']:.2f}M in Q4 2024, representing {top_commodity['share_q4']:.1f}% of total exports"
        })
    
    # Generate Recommendations
    recommendations.append({
        'category': 'Trade Diversification',
        'priority': 'High',
        'recommendation': 'Develop strategies to diversify export markets beyond the top 5 destinations',
        'rationale': 'Current concentration increases vulnerability to market-specific shocks',
        'actions': [
            'Identify and target emerging markets in Asia and Europe',
            'Strengthen trade relations with West African markets',
            'Participate in international trade fairs and exhibitions',
            'Develop market intelligence capabilities'
        ]
    })
    
    recommendations.append({
        'category': 'Export Promotion',
        'priority': 'High',
        'recommendation': 'Enhance support for high-growth export commodities',
        'rationale': 'Focus resources on sectors with proven growth potential',
        'actions': [
            'Provide targeted incentives for top-performing commodities',
            'Invest in quality certification and standards compliance',
            'Develop supply chain infrastructure for key exports',
            'Create sector-specific export promotion programs'
        ]
    })
    
    recommendations.append({
        'category': 'Import Substitution',
        'priority': 'Medium',
        'recommendation': 'Identify opportunities for import substitution in key commodity categories',
        'rationale': 'Reduce trade deficit by developing domestic production capacity',
        'actions': [
            'Analyze top import commodities for local production potential',
            'Assess feasibility of domestic manufacturing for selected goods',
            'Provide incentives for import-substituting industries',
            'Develop backward integration strategies'
        ]
    })
    
    recommendations.append({
        'category': 'Regional Integration',
        'priority': 'High',
        'recommendation': 'Strengthen EAC trade integration and explore new regional partnerships',
        'rationale': 'EAC represents a significant portion of Rwanda\'s export market',
        'actions': [
            'Enhance participation in EAC trade protocols',
            'Improve cross-border trade infrastructure',
            'Develop regional value chains',
            'Strengthen trade facilitation measures'
        ]
    })
    
    recommendations.append({
        'category': 'Trade Infrastructure',
        'priority': 'Medium',
        'recommendation': 'Invest in trade infrastructure and logistics capabilities',
        'rationale': 'Efficient logistics are crucial for maintaining export competitiveness',
        'actions': [
            'Upgrade port and transportation facilities',
            'Implement trade facilitation technologies',
            'Develop warehousing and storage facilities',
            'Improve customs clearance processes'
        ]
    })
    
    return insights, recommendations

# Generate insights and recommendations
insights, recommendations = generate_insights_and_recommendations()

print("🔍 Key Insights and Recommendations")
print("=" * 50)

print(f"\n📊 Total Insights Generated: {len(insights)}")
print(f"📋 Total Recommendations: {len(recommendations)}")

print(f"\n💡 KEY INSIGHTS:")
print("-" * 30)
for insight in insights:
    print(f"\n{insight['category'].upper()}:")
    print(f"   {insight['insight']}")
    print(f"   💬 {insight['details']}")

print(f"\n🎯 STRATEGIC RECOMMENDATIONS:")
print("-" * 40)
for rec in recommendations:
    print(f"\n{rec['category']} (Priority: {rec['priority']}):")
    print(f"   🎯 {rec['recommendation']}")
    print(f"   📝 Rationale: {rec['rationale']}")
    print(f"   ✅ Actions:")
    for action in rec['actions']:
        print(f"      • {action}")

🔍 Key Insights and Recommendations

📊 Total Insights Generated: 1
📋 Total Recommendations: 5

💡 KEY INSIGHTS:
------------------------------

LEADING EXPORT:
   Other commodities & transactions, n.e.s is Rwanda's leading export commodity
   💬 Generated $428.15M in Q4 2024, representing 63.2% of total exports

🎯 STRATEGIC RECOMMENDATIONS:
----------------------------------------

Trade Diversification (Priority: High):
   🎯 Develop strategies to diversify export markets beyond the top 5 destinations
   📝 Rationale: Current concentration increases vulnerability to market-specific shocks
   ✅ Actions:
      • Identify and target emerging markets in Asia and Europe
      • Strengthen trade relations with West African markets
      • Participate in international trade fairs and exhibitions
      • Develop market intelligence capabilities

Export Promotion (Priority: High):
   🎯 Enhance support for high-growth export commodities
   📝 Rationale: Focus resources on sectors with proven growth p

In [23]:
# Create insights dashboard
def create_insights_dashboard(insights, recommendations):
    """
    Create a visual dashboard of key insights and recommendations
    """
    # Categorize insights by type
    insight_types = {}
    for insight in insights:
        insight_type = insight['type']
        if insight_type not in insight_types:
            insight_types[insight_type] = 0
        insight_types[insight_type] += 1
    
    # Categorize recommendations by priority
    priority_counts = {}
    for rec in recommendations:
        priority = rec['priority']
        if priority not in priority_counts:
            priority_counts[priority] = 0
        priority_counts[priority] += 1
    
    # Create insights dashboard
    fig_dashboard = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Insights by Category',
            'Insights by Type',
            'Recommendations by Priority',
            'Action Items Summary'
        ),
        specs=[[{"type": "bar"}, {"type": "pie"}],
               [{"type": "bar"}, {"type": "table"}]]
    )
    
    # 1. Insights by Category
    categories = [insight['category'] for insight in insights]
    category_counts = {}
    for cat in categories:
        if cat not in category_counts:
            category_counts[cat] = 0
        category_counts[cat] += 1
    
    sorted_categories = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)
    
    fig_dashboard.add_trace(
        go.Bar(
            x=[cat[1] for cat in sorted_categories],
            y=[cat[0] for cat in sorted_categories],
            orientation='h',
            name='Insights by Category',
            marker_color='#2E86AB'
        ),
        row=1, col=1
    )
    
    # 2. Insights by Type
    fig_dashboard.add_trace(
        go.Pie(
            labels=list(insight_types.keys()),
            values=list(insight_types.values()),
            name='Insights by Type',
            textinfo='label+value',
            textposition='inside'
        ),
        row=1, col=2
    )
    
    # 3. Recommendations by Priority
    fig_dashboard.add_trace(
        go.Bar(
            x=list(priority_counts.values()),
            y=list(priority_counts.keys()),
            orientation='h',
            name='Recommendations by Priority',
            marker_color=['#F18F01' if p == 'High' else '#2E86AB' if p == 'Medium' else '#A23B72' for p in priority_counts.keys()]
        ),
        row=2, col=1
    )
    
    # 4. Action Items Summary
    total_actions = sum(len(rec['actions']) for rec in recommendations)
    high_priority_actions = sum(len(rec['actions']) for rec in recommendations if rec['priority'] == 'High')
    medium_priority_actions = sum(len(rec['actions']) for rec in recommendations if rec['priority'] == 'Medium')
    
    # Create summary table
    summary_data = [
        ['Total Recommendations', len(recommendations)],
        ['Total Action Items', total_actions],
        ['High Priority Actions', high_priority_actions],
        ['Medium Priority Actions', medium_priority_actions],
        ['Insights Generated', len(insights)]
    ]
    
    fig_dashboard.add_trace(
        go.Table(
            header=dict(values=['Metric', 'Value']),
            cells=dict(values=list(zip(*summary_data)))
        ),
        row=2, col=2
    )
    
    # Update layout
    fig_dashboard.update_layout(
        height=800,
        title_text="Rwanda Trade Analysis - Insights Dashboard",
        title_x=0.5,
        showlegend=False
    )
    
    return fig_dashboard

# Create and display insights dashboard
insights_dashboard = create_insights_dashboard(insights, recommendations)
insights_dashboard.show()

print("✅ Insights dashboard created successfully")

✅ Insights dashboard created successfully


In [24]:
# 7. Summary and Conclusion

This comprehensive analysis of Rwanda's trade data provides valuable insights into the country's external trade performance, patterns, and opportunities.

SyntaxError: invalid syntax (1108337441.py, line 3)

In [25]:
# Generate comprehensive summary report
def generate_summary_report():
    """
    Generate a comprehensive summary report of the analysis
    """
    summary = {
        'analysis_overview': {
            'title': 'Rwanda Trade Data Analysis - Q4 2024',
            'period': 'Q1 2022 - Q4 2024',
            'data_source': 'NISR Trade Report Annex Tables',
            'analysis_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
            'sheets_analyzed': len(cleaned_data)
        },
        'key_metrics': {},
        'top_performers': {},
        'challenges': [],
        'opportunities': [],
        'recommendations_summary': []
    }
    
    # Key Metrics
    if 'Graph Overall' in cleaned_data:
        latest = cleaned_data['Graph Overall'].iloc[-1]
        summary['key_metrics'] = {
            'total_exports_q4_2024': f"${latest['exports']:.2f}M",
            'total_imports_q4_2024': f"${latest['imports']:.2f}M",
            'total_reexports_q4_2024': f"${latest['reexports']:.2f}M",
            'trade_balance_q4_2024': f"${latest['trade_balance']:.2f}M",
            'total_trade_volume': f"${latest['total_trade']:.2f}M"
        }
    
    # Top Performers
    if 'ExportCountry' in cleaned_data:
        top_export = cleaned_data['ExportCountry'].loc[cleaned_data['ExportCountry']['q4_2024'].idxmax()]
        summary['top_performers']['top_export_destination'] = {
            'country': top_export['country'],
            'value': f"${top_export['q4_2024']:.2f}M",
            'share': f"{top_export['share_q4']:.2f}%"
        }
    
    if 'ImportCountry' in cleaned_data:
        top_import = cleaned_data['ImportCountry'].loc[cleaned_data['ImportCountry']['q4_2024'].idxmax()]
        summary['top_performers']['top_import_source'] = {
            'country': top_import['country'],
            'value': f"${top_import['q4_2024']:.2f}M",
            'share': f"{top_import['share_q4']:.2f}%"
        }
    
    if 'ExportsCommodity' in cleaned_data:
        top_commodity = cleaned_data['ExportsCommodity'].loc[cleaned_data['ExportsCommodity']['q4_2024'].idxmax()]
        summary['top_performers']['top_export_commodity'] = {
            'commodity': top_commodity['description'],
            'sitc_section': top_commodity['sitc_section'],
            'value': f"${top_commodity['q4_2024']:.2f}M",
            'share': f"{top_commodity['share_q4']:.2f}%"
        }
    
    # Challenges
    if 'Graph Overall' in cleaned_data:
        latest = cleaned_data['Graph Overall'].iloc[-1]
        if latest['trade_balance'] < 0:
            deficit_ratio = abs(latest['trade_balance']) / latest['imports'] * 100
            summary['challenges'].append({
                'issue': 'Persistent Trade Deficit',
                'details': f"Trade deficit of ${latest['trade_balance']:.2f}M represents {deficit_ratio:.1f}% of imports",
                'impact': 'High'
            })
    
    if 'ExportCountry' in cleaned_data:
        export_countries = cleaned_data['ExportCountry']
        top_5_share = export_countries.nlargest(5, 'q4_2024')['q4_2024'].sum() / export_countries['q4_2024'].sum() * 100
        if top_5_share > 80:
            summary['challenges'].append({
                'issue': 'Market Concentration Risk',
                'details': f"Top 5 export destinations account for {top_5_share:.1f}% of total exports",
                'impact': 'Medium'
            })
    
    # Opportunities
    if 'EAC' in cleaned_data:
        eac_df = cleaned_data['EAC']
        total_eac = eac_df['q4_2024'].sum()
        if 'Graph Overall' in cleaned_data:
            total_exports = cleaned_data['Graph Overall'].iloc[-1]['exports']
            eac_share = (total_eac / total_exports) * 100
            summary['opportunities'].append({
                'opportunity': 'Regional Market Expansion',
                'details': f"EAC market represents {eac_share:.1f}% of exports with growth potential",
                'potential': 'High'
            })
    
    if 'ExportsCommodity' in cleaned_data:
        export_commodities = cleaned_data['ExportsCommodity']
        high_growth = export_commodities[export_commodities['growth_yoy'] > 20].nlargest(3, 'growth_yoy')
        for _, commodity in high_growth.iterrows():
            summary['opportunities'].append({
                'opportunity': f"High-Growth Commodity: {commodity['description']}",
                'details': f"YoY growth of {commodity['growth_yoy']:.1f}% in Q4 2024",
                'potential': 'High'
            })
    
    # Recommendations Summary
    for rec in recommendations:
        summary['recommendations_summary'].append({
            'category': rec['category'],
            'priority': rec['priority'],
            'summary': rec['recommendation'],
            'action_count': len(rec['actions'])
        })
    
    return summary

# Generate and display summary report
summary_report = generate_summary_report()

print("📋 COMPREHENSIVE ANALYSIS SUMMARY")
print("=" * 50)

print(f"\n📊 ANALYSIS OVERVIEW:")
print(f"   Title: {summary_report['analysis_overview']['title']}")
print(f"   Period: {summary_report['analysis_overview']['period']}")
print(f"   Data Source: {summary_report['analysis_overview']['data_source']}")
print(f"   Analysis Date: {summary_report['analysis_overview']['analysis_date']}")
print(f"   Sheets Analyzed: {summary_report['analysis_overview']['sheets_analyzed']}")

print(f"\n💰 KEY METRICS (Q4 2024):")
for metric, value in summary_report['key_metrics'].items():
    print(f"   {metric.replace('_', ' ').title()}: {value}")

print(f"\n🏆 TOP PERFORMERS:")
for category, performer in summary_report['top_performers'].items():
    print(f"   {category.replace('_', ' ').title()}:")
    for key, value in performer.items():
        print(f"      {key.replace('_', ' ').title()}: {value}")

print(f"\n⚠️ KEY CHALLENGES:")
for challenge in summary_report['challenges']:
    print(f"   {challenge['issue']} (Impact: {challenge['impact']})")
    print(f"      {challenge['details']}")

print(f"\n🚀 OPPORTUNITIES:")
for opportunity in summary_report['opportunities']:
    print(f"   {opportunity['opportunity']} (Potential: {opportunity['potential']})")
    print(f"      {opportunity['details']}")

print(f"\n🎯 RECOMMENDATIONS SUMMARY:")
for rec in summary_report['recommendations_summary']:
    print(f"   {rec['category']} (Priority: {rec['priority']})")
    print(f"      {rec['summary']}")
    print(f"      Actions: {rec['action_count']} items")

print(f"\n📈 ANALYSIS COMPLETED SUCCESSFULLY!")
print(f"   Total insights generated: {len(insights)}")
print(f"   Total recommendations: {len(recommendations)}")
print(f"   Data visualizations created: 6 comprehensive charts")
print(f"   Analysis covers: {len(cleaned_data)} Excel sheets")
print(f"   Time period: 12 quarters (2022-2024)")
print(f"   Trade volume analyzed: ~${cleaned_data['Graph Overall'].iloc[-1]['total_trade']:.0f}M")

print("\n" + "=" * 50)
print("🎉 RWANDA TRADE ANALYSIS COMPLETE")
print("=" * 50)

📋 COMPREHENSIVE ANALYSIS SUMMARY

📊 ANALYSIS OVERVIEW:
   Title: Rwanda Trade Data Analysis - Q4 2024
   Period: Q1 2022 - Q4 2024
   Data Source: NISR Trade Report Annex Tables
   Analysis Date: 2025-09-25 10:33:39
   Sheets Analyzed: 12

💰 KEY METRICS (Q4 2024):
   Total Exports Q4 2024: $0.00M
   Total Imports Q4 2024: $0.00M
   Total Reexports Q4 2024: $0.00M
   Trade Balance Q4 2024: $0.00M
   Total Trade Volume: $0.00M

🏆 TOP PERFORMERS:
   Top Export Destination:
      Country: China
      Value: $20.43M
      Share: 3.02%
   Top Import Source:
      Country: Kenya
      Value: $211.22M
      Share: 12.96%
   Top Export Commodity:
      Commodity: Other commodities & transactions, n.e.s
      Sitc Section: 9
      Value: $428.15M
      Share: 63.20%

⚠️ KEY CHALLENGES:

🚀 OPPORTUNITIES:
   Regional Market Expansion (Potential: High)
      EAC market represents nan% of exports with growth potential
   High-Growth Commodity: Animals and vegetable oils, fats & waxes (Potential: Hig

In [26]:
# Save comprehensive analysis results - CHANGED to JSON format
import json
from datetime import datetime

def save_analysis_results():
    """
    Save all analysis results to JSON files for future reference
    """
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Prepare comprehensive results
    results = {
        'metadata': {
            'analysis_timestamp': timestamp,
            'data_source': EXCEL_FILE,
            'sheets_analyzed': len(cleaned_data),
            'analysis_version': '2.0',
            'analyst': 'Rwanda Export Explorer AI Analysis'
        },
        'summary_report': summary_report,
        'insights': insights,
        'recommendations': recommendations,
        'data_statistics': {
            'total_sheets': len(cleaned_data),
            'total_data_points': sum(len(df) for df in cleaned_data.values()),
            'quarters_covered': 12,
            'countries_analyzed': len(cleaned_data.get('ExportCountry', [])) + len(cleaned_data.get('ImportCountry', [])) + len(cleaned_data.get('ReexportsCountry', [])),
            'commodities_analyzed': len(cleaned_data.get('ExportsCommodity', [])) + len(cleaned_data.get('ImportsCommodity', [])) + len(cleaned_data.get('ReexportsCommodity', []))
        }
    }
    
    # Save main results - CHANGED to JSON
    output_file = f"{OUTPUT_DIR}comprehensive_analysis_results_{timestamp}.json"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False, default=str)
    
    print(f"✅ Analysis results saved to: {output_file}")
    
    # Save individual sheet data - CHANGED to JSON
    for sheet_name, df in cleaned_data.items():
        sheet_file = f"{OUTPUT_DIR}sheet_{sheet_name.replace(' ', '_').lower()}_{timestamp}.json"
        df.to_json(sheet_file, orient='records', indent=2)
        print(f"✅ {sheet_name} data saved to: {sheet_file}")
    
    return output_file

# Save all results
main_results_file = save_analysis_results()
print(f"\n📁 All analysis files saved successfully!")
print(f"   Main results: {main_results_file}")
print(f"   Individual sheets: {len(cleaned_data)} JSON files")
print(f"   Total files created: {len(cleaned_data) + 1}")
print(f"\n🎉 ANALYSIS COMPLETE - All issues fixed!")

✅ Analysis results saved to: data/processed/comprehensive_analysis_results_20250925_103357.json
✅ Graph Overall data saved to: data/processed/sheet_graph_overall_20250925_103357.json
✅ Graph EAC data saved to: data/processed/sheet_graph_eac_20250925_103357.json
✅ EAC data saved to: data/processed/sheet_eac_20250925_103357.json
✅ Total trade with the World data saved to: data/processed/sheet_total_trade_with_the_world_20250925_103357.json
✅ Regional blocks data saved to: data/processed/sheet_regional_blocks_20250925_103357.json
✅ Trade by continents data saved to: data/processed/sheet_trade_by_continents_20250925_103357.json
✅ ExportCountry data saved to: data/processed/sheet_exportcountry_20250925_103357.json
✅ ImportCountry data saved to: data/processed/sheet_importcountry_20250925_103357.json
✅ ReexportsCountry data saved to: data/processed/sheet_reexportscountry_20250925_103357.json
✅ ExportsCommodity data saved to: data/processed/sheet_exportscommodity_20250925_103357.json
✅ Import